In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima,arima
from itertools import product
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("train.csv",parse_dates=['date'])
test = pd.read_csv("test.csv",parse_dates=['date'])
oil = pd.read_csv("oil.csv",parse_dates=['date'])
holiday = pd.read_csv("holidays_events.csv",parse_dates=['date'])
transactions = pd.read_csv("transactions.csv",parse_dates=['date'])
sample = pd.read_csv("sample_submission.csv")
stores = pd.read_csv("stores.csv")

In [ ]:
family_group=[]
for i in range(33,66):
    family_group.append(train.iloc[i,3])

In [ ]:
result=pd.concat([train,test])
result

In [ ]:

r1=pd.merge(result,stores,how="left",on=["store_nbr"])
r2=pd.merge(r1,transactions,how="left",on=["date","store_nbr"])
Data=pd.merge(r2,oil,how="left",on=["date"])
Data

In [ ]:
Data.isna().sum()

In [ ]:
Data.fillna(value={'sales':0.0},inplace=True)
Data.fillna(value={'transactions':0.0},inplace=True)
Data.fillna(value={'dcoilwtico':0.0},inplace=True)

In [ ]:
Data.isna().sum()

In [ ]:
Data.head(10)


In [ ]:
Data.set_index("store_nbr",inplace=True)

In [ ]:
store={}
store_family={}
family_group

In [ ]:
for i in range(1,55):
    store[i]=df.loc[i]
    store[i].set_index("family",inplace=True)
    for j in range(0,33):
        store_family[i,j]=store[i].loc[family_group[j]]
        store_family[i,j].set_index("date",inplace=True)

In [ ]:
df=store_family[1,0]

In [ ]:
df["sales"].plot()

In [ ]:
p = range(0, 4, 1)
d = 1
q = range(0, 4, 1)
P = range(0, 4, 1)
D = 1
Q = range(0, 4, 1)
s = 4
parameters = product(p, q, P, Q)
parameters_list = list(parameters)
print(len(parameters_list))

In [ ]:
def transactionsforecast(df,array,d):
    data=df['transactions']
    train_transactions_data=data.copy()
    train_transactions_data.loc[-16:]=np.nan
    test_transactions_exog=df['dcoilwtico'].iloc[-16:]
    model=SARIMAX(endog=train_transactions_data,order=(0, 0, 1), seasonal_order=(0, 0, 1, 12),exog=df['dcoilwtico']).fit()
    start=datetime(2017,8,16)
    end=datetime(2017,8,31)
    forecast=model.predict(start=start ,end=end,exog=test_transactions_exog)
    exog_predict=df[array][-16:]
    if d==1:
        exog_predict['transactions'][-16:]=forecast1
        return exog_predict
    else:
        return exog_predict

In [ ]:
def optimize_SARIMAX(parameters_list, d, D, s, endog,df,exoglist,c):
    results = []
    for param in tqdm_notebook(parameters_list):
        try:
            if exoglist:
                model = SARIMAX(endog=endog,exog=df[exoglist], order=(param[0], d, param[1]),k_exog=c, seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
            else:
                model = SARIMAX(endog=endog, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        results.append([param, aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q,P,Q)', 'AIC']
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
    param=result_df['(p,q,P,Q)'][0]
    return param

In [ ]:
def sarimax(df):
    df.sort_values('id')
    correlation=df.corr(method='pearson')
    array=[]
    c=0
    d=0
    if correlation['sales']['onpromotion']>0.5 and correlation['sales']['onpromotion']<1:
        array+=['onpromotion']
        c+=1
    if correlation['sales']['transactions']>0.5 and correlation['sales']['transactions']<1:
        array+=['transactions']
        c+=1
        d+=1
    if correlation['sales']['dcoilwtico']>0.5 and correlation['sales']['dcoilwtico']<1:
        array+=['dcoilwtico']
        c+=1
    D=arima.nsdiffs(df['sales'][:-16],12)
    d=arima.ndiffs(df['sales'][:-16])
    s=12
    param=optimize_SARIMAX(parameters_list, d, D, s, df['sales'],df,array,c)
    D=arima.nsdiffs(df['sales'][:-16],12)
    d=arima.ndiffs(df['sales'][:-16])
    if array:        
        model = SARIMAX(endog=df['sales'],exog=df[array], order=(param[0], d, param[1]),k_exog=c, seasonal_order=(param[2], D, param[3], s)).fit()
    else:
        model = SARIMAX(endog=df['sales'], order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
    exog_predict=transactionsforecast(df,array,d)
    start=datetime(2017,8,16)
    end=datetime(2017,8,31)
    if array:
        forecast=model.predict(start=start ,end=end,exog=exog_predict)
    else:
        forecast=model.predict(start=start ,end=end)
    
   ''' error=np.sqrt(mean_squared_error(forecast,df["sales"]))
    print("error:",error)'''
    pred=forecast.tail(16)
    data=df[['id','sales']].tail(16)
    for i in range(0,16):
        data['sales'][i]=pred[i]
    return data

In [ ]:
forecast=pd.DataFrame()
for i in range(1,55):
    for j in range(0,33):
        data=sarimax(store_family[i,j])
        forecast=pd.concat([forecast,data])
forecast.sort_values('id')
forecast.to_csv("submission.csv", index = False)
forecast